In [7]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime

In [8]:
response = requests.get("https://lamanzanamordida.net/tutoriales/iphone/lanzamiento-iphone-fechas-todos-modelos/")
soup = BeautifulSoup(response.text, 'html.parser')


In [9]:
 
models = pd.DataFrame([tag.text for tag in soup.find_all('h3')]).rename(columns={0:"Model"}).reset_index()

In [10]:
models

,index,Model
0,0,iPhone (original)
1,1,iPhone 3G
2,2,iPhone 3GS
3,3,iPhone 4
4,4,iPhone 4s
5,5,iPhone 5
6,6,iPhone 5s
7,7,iPhone 5c
8,8,iPhone 6
9,9,iPhone 6 Plus


In [11]:
data_raw = [tag.text for tag in soup.select('ul li')]
#data_raw[53:151]

In [12]:
dates = pd.DataFrame(data_raw[53:151])
dates = dates[0].str.split(":",expand=True)
dates = dates[dates[0].str.contains("Fecha de presentación")]

dates = dates.reset_index(drop=True).drop(columns=[0]).rename(columns={1:"Date"})

dates = dates.reset_index()

In [13]:
dates

,index,Date
0,0,9 de enero de 2007.
1,1,9 de junio de 2008.
2,2,8 de junio de 2009.
3,3,7 de junio de 2010.
4,4,4 de octubre de 2011.
5,5,12 de septiembre de 2012.
6,6,10 de septiembre de 2013..
7,7,10 de septiembre de 2013.
8,8,9 de septiembre de 2014.
9,9,9 de septiembre de 2014.


In [14]:
intro =  pd.merge(models,dates,on="index").drop(columns=["index"]).set_index("Model")
intro.head(2)

,Date
Model,
iPhone (original),9 de enero de 2007.
iPhone 3G,9 de junio de 2008.


In [15]:
intro = intro.replace(r"de|\.|\..|\s","",regex=True)

In [16]:
intro['ano'] = intro['Date'].str.extract('(\d{4}$)')
intro['mes'] = intro['Date'].str.extract('(\D+)')
intro['dia'] = intro['Date'].str.extract('(\d+)')
dict(intro.mes)

intro = intro.replace({'mes':{'enero': '01',
                              'marzo': '03',
                              'abril': '04',
                              'junio': '06',    
                              'septiembre': '09',
                              'octubre': '10'}})

intro['Date'] = intro["ano"]+"-"+intro["mes"]+"-"+intro["dia"]
intro = intro.drop(columns=["ano","mes","dia"])

In [17]:
intro["Date"] = intro["Date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [18]:
intro

,Date
Model,
iPhone (original),2007-01-09
iPhone 3G,2008-06-09
iPhone 3GS,2009-06-08
iPhone 4,2010-06-07
iPhone 4s,2011-10-04
iPhone 5,2012-09-12
iPhone 5s,2013-09-10
iPhone 5c,2013-09-10
iPhone 6,2014-09-09


In [19]:
intro.to_csv("intro_dates.csv")

In [25]:
dateFrom = "2019-12-05"
dateTo = "2019-05-05"

def import_model(dateFrom,dateTo,intro):
     return intro[(intro["Date"]>= dateTo) & (intro["Date"] <= dateFrom)]
import_model(dateFrom,dateTo,intro)

,Date
Model,
iPhone 11,2019-09-10
iPhone 11 Pro,2019-09-10
iPhone 11 Pro Max,2019-09-10


In [29]:
modelos = pd.read_csv("intro_dates.csv")
x = modelos[(modelos["Date"]>= dateTo) & (modelos["Date"] <= dateFrom)]
x

,Model,Date
21,iPhone 11,2019-09-10
22,iPhone 11 Pro,2019-09-10
23,iPhone 11 Pro Max,2019-09-10
